### Creating a database

In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

Project_Root_Directory = Path(os.getcwd()).resolve().parents[1]  # adjust as needed

# Database configuration
DB_DIRECTORY = os.getenv("DB_DIRECTORY", "src/db")
DB_NAME = 'showroom_management.db'
DB_PATH = Project_Root_Directory / DB_DIRECTORY / DB_NAME

# # Create connection
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

print("📁 Database connection established successfully!")
print(f"Database: {DB_NAME}")


📁 Database connection established successfully!
Database: showroom_management.db


In [2]:
def create_tables():
    """Create all required tables for showroom management"""
    
    # 1. Showrooms Table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS showrooms (
        showroom_id INTEGER PRIMARY KEY AUTOINCREMENT,
        showroom_name TEXT NOT NULL,
        brand TEXT NOT NULL,
        address TEXT NOT NULL,
        city TEXT NOT NULL,
        state TEXT NOT NULL,
        phone TEXT NOT NULL,
        email TEXT NOT NULL,
        manager_name TEXT NOT NULL,
        opening_date DATE,
        status TEXT DEFAULT 'Active'
    )
    ''')
    
    # 2. Vehicles Table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS vehicles (
        vehicle_id INTEGER PRIMARY KEY AUTOINCREMENT,
        brand TEXT NOT NULL,
        model_name TEXT NOT NULL,
        variant TEXT,
        vehicle_type TEXT NOT NULL,
        engine_capacity TEXT,
        fuel_type TEXT,
        transmission TEXT,
        color_options TEXT,
        base_price REAL NOT NULL,
        launch_date DATE,
        status TEXT DEFAULT 'Active'
    )
    ''')
    
    # 3. Customers Table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS customers (
        customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        phone TEXT UNIQUE NOT NULL,
        email TEXT UNIQUE,
        address TEXT,
        city TEXT,
        state TEXT,
        date_of_birth DATE,
        registration_date DATE DEFAULT CURRENT_DATE,
        customer_type TEXT DEFAULT 'New'
    )
    ''')
    
    # 4. Employees Table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
        showroom_id INTEGER,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        position TEXT NOT NULL,
        phone TEXT,
        email TEXT,
        hire_date DATE,
        salary REAL,
        commission_rate REAL DEFAULT 0.02,
        status TEXT DEFAULT 'Active',
        FOREIGN KEY (showroom_id) REFERENCES showrooms (showroom_id)
    )
    ''')
    
    # 5. Inventory Table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS inventory (
        inventory_id INTEGER PRIMARY KEY AUTOINCREMENT,
        showroom_id INTEGER,
        vehicle_id INTEGER,
        stock_quantity INTEGER DEFAULT 0,
        reserved_quantity INTEGER DEFAULT 0,
        available_quantity INTEGER DEFAULT 0,
        reorder_level INTEGER DEFAULT 5,
        last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (showroom_id) REFERENCES showrooms (showroom_id),
        FOREIGN KEY (vehicle_id) REFERENCES vehicles (vehicle_id)
    )
    ''')
    
    # 6. Sales Table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS sales (
        sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
        showroom_id INTEGER,
        customer_id INTEGER,
        vehicle_id INTEGER,
        salesperson_id INTEGER,
        sale_date DATE,
        sale_price REAL,
        discount_amount REAL DEFAULT 0,
        final_amount REAL,
        payment_method TEXT,
        delivery_date DATE,
        sale_status TEXT DEFAULT 'Completed',
        FOREIGN KEY (showroom_id) REFERENCES showrooms (showroom_id),
        FOREIGN KEY (customer_id) REFERENCES customers (customer_id),
        FOREIGN KEY (vehicle_id) REFERENCES vehicles (vehicle_id),
        FOREIGN KEY (salesperson_id) REFERENCES employees (employee_id)
    )
    ''')
    
    # 7. Test Drives Table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS test_drives (
        test_drive_id INTEGER PRIMARY KEY AUTOINCREMENT,
        customer_id INTEGER,
        vehicle_id INTEGER,
        showroom_id INTEGER,
        scheduled_date DATE,
        scheduled_time TIME,
        duration INTEGER DEFAULT 30,
        status TEXT DEFAULT 'Scheduled',
        feedback TEXT,
        converted_to_sale BOOLEAN DEFAULT 0,
        FOREIGN KEY (customer_id) REFERENCES customers (customer_id),
        FOREIGN KEY (vehicle_id) REFERENCES vehicles (vehicle_id),
        FOREIGN KEY (showroom_id) REFERENCES showrooms (showroom_id)
    )
    ''')
    
    # 8. Service Records Table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS service_records (
        service_id INTEGER PRIMARY KEY AUTOINCREMENT,
        customer_id INTEGER,
        vehicle_id INTEGER,
        showroom_id INTEGER,
        service_date DATE,
        service_type TEXT,
        description TEXT,
        cost REAL,
        technician_id INTEGER,
        next_service_due DATE,
        FOREIGN KEY (customer_id) REFERENCES customers (customer_id),
        FOREIGN KEY (vehicle_id) REFERENCES vehicles (vehicle_id),
        FOREIGN KEY (showroom_id) REFERENCES showrooms (showroom_id),
        FOREIGN KEY (technician_id) REFERENCES employees (employee_id)
    )
    ''')
    
    # 9. Expenses Table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS expenses (
        expense_id INTEGER PRIMARY KEY AUTOINCREMENT,
        showroom_id INTEGER,
        expense_category TEXT,
        description TEXT,
        amount REAL,
        expense_date DATE,
        approved_by TEXT,
        receipt_number TEXT,
        FOREIGN KEY (showroom_id) REFERENCES showrooms (showroom_id)
    )
    ''')
    
    # 10. Targets Table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS targets (
        target_id INTEGER PRIMARY KEY AUTOINCREMENT,
        showroom_id INTEGER,
        employee_id INTEGER,
        target_period TEXT,
        target_type TEXT,
        target_value REAL,
        achieved_value REAL DEFAULT 0,
        start_date DATE,
        end_date DATE,
        FOREIGN KEY (showroom_id) REFERENCES showrooms (showroom_id),
        FOREIGN KEY (employee_id) REFERENCES employees (employee_id)
    )
    ''')
    
    conn.commit()
    print("✅ All tables created successfully!")

# Execute the function
create_tables()


✅ All tables created successfully!


In [3]:
def insert_showroom_data():
    """Insert sample showroom data"""
    showroom_data = [
        ('Ford Downtown', 'Ford', '123 Main Street', 'Mumbai', 'Maharashtra', '+91-9876543210', 'mumbai.ford@example.com', 'Rajesh Kumar', '2020-01-15', 'Active'),
        ('Ford Highway', 'Ford', '456 Highway Road', 'Delhi', 'Delhi', '+91-9876543211', 'delhi.ford@example.com', 'Priya Sharma', '2019-08-20', 'Active'),
        ('Ford Central', 'Ford', '789 Central Plaza', 'Bangalore', 'Karnataka', '+91-9876543212', 'bangalore.ford@example.com', 'Amit Patel', '2021-03-10', 'Active'),
        ('Ford Elite', 'Ford', '321 Elite Avenue', 'Chennai', 'Tamil Nadu', '+91-9876543213', 'chennai.ford@example.com', 'Sunita Reddy', '2020-11-05', 'Active'),
        ('Ford Premium', 'Ford', '654 Premium Street', 'Hyderabad', 'Telangana', '+91-9876543214', 'hyderabad.ford@example.com', 'Vikram Singh', '2022-01-12', 'Active'),
        ('Royal Enfield Classic', 'Royal Enfield', '111 Bike Street', 'Mumbai', 'Maharashtra', '+91-8765432100', 'mumbai.re@example.com', 'Arjun Mehta', '2019-05-15', 'Active'),
        ('Royal Enfield Thunder', 'Royal Enfield', '222 Thunder Lane', 'Delhi', 'Delhi', '+91-8765432101', 'delhi.re@example.com', 'Kavya Joshi', '2020-02-28', 'Active'),
        ('Royal Enfield Heritage', 'Royal Enfield', '333 Heritage Road', 'Bangalore', 'Karnataka', '+91-8765432102', 'bangalore.re@example.com', 'Rohit Gupta', '2021-07-20', 'Active'),
        ('Royal Enfield Vintage', 'Royal Enfield', '444 Vintage Plaza', 'Chennai', 'Tamil Nadu', '+91-8765432103', 'chennai.re@example.com', 'Neha Agarwal', '2020-09-18', 'Active'),
        ('Royal Enfield Royal', 'Royal Enfield', '555 Royal Avenue', 'Hyderabad', 'Telangana', '+91-8765432104', 'hyderabad.re@example.com', 'Sanjay Rao', '2022-04-10', 'Active'),
        ('Ford Express', 'Ford', '777 Express Way', 'Pune', 'Maharashtra', '+91-9876543215', 'pune.ford@example.com', 'Deepika Nair', '2021-12-01', 'Active'),
        ('Ford Luxury', 'Ford', '888 Luxury Lane', 'Ahmedabad', 'Gujarat', '+91-9876543216', 'ahmedabad.ford@example.com', 'Manoj Tiwari', '2020-06-15', 'Active'),
        ('Royal Enfield Adventure', 'Royal Enfield', '666 Adventure Street', 'Pune', 'Maharashtra', '+91-8765432105', 'pune.re@example.com', 'Anita Desai', '2021-01-25', 'Active'),
        ('Royal Enfield Storm', 'Royal Enfield', '777 Storm Road', 'Ahmedabad', 'Gujarat', '+91-8765432106', 'ahmedabad.re@example.com', 'Ravi Kumar', '2020-10-30', 'Active'),
        ('Ford Metro', 'Ford', '999 Metro Plaza', 'Kolkata', 'West Bengal', '+91-9876543217', 'kolkata.ford@example.com', 'Shreya Banerjee', '2022-02-14', 'Active'),
        ('Royal Enfield Legend', 'Royal Enfield', '888 Legend Lane', 'Kolkata', 'West Bengal', '+91-8765432107', 'kolkata.re@example.com', 'Abhishek Ghosh', '2021-11-08', 'Active'),
        ('Ford Platinum', 'Ford', '101 Platinum Street', 'Jaipur', 'Rajasthan', '+91-9876543218', 'jaipur.ford@example.com', 'Pooja Agarwal', '2021-05-20', 'Active'),
        ('Royal Enfield Crown', 'Royal Enfield', '202 Crown Avenue', 'Jaipur', 'Rajasthan', '+91-8765432108', 'jaipur.re@example.com', 'Karan Sharma', '2020-12-03', 'Active'),
        ('Ford Supreme', 'Ford', '303 Supreme Plaza', 'Indore', 'Madhya Pradesh', '+91-9876543219', 'indore.ford@example.com', 'Ritika Jain', '2022-03-25', 'Active'),
        ('Royal Enfield Spirit', 'Royal Enfield', '404 Spirit Road', 'Indore', 'Madhya Pradesh', '+91-8765432109', 'indore.re@example.com', 'Gaurav Pandey', '2021-08-17', 'Active')
    ]
    
    cursor.executemany('''
        INSERT INTO showrooms (showroom_name, brand, address, city, state, phone, email, manager_name, opening_date, status)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', showroom_data)
    
    conn.commit()
    print("✅ Showroom data inserted successfully!")

insert_showroom_data()


✅ Showroom data inserted successfully!


In [4]:
def insert_vehicle_data():
    """Insert sample vehicle data"""
    vehicle_data = [
        ('Ford', 'EcoSport', 'Titanium', 'Car', '1.5L', 'Petrol', 'Manual', 'White, Black, Silver, Red', 1150000, '2021-01-15', 'Active'),
        ('Ford', 'EcoSport', 'Titanium Plus', 'Car', '1.5L', 'Petrol', 'Automatic', 'White, Black, Silver, Blue', 1280000, '2021-01-15', 'Active'),
        ('Ford', 'Figo', 'Titanium', 'Car', '1.2L', 'Petrol', 'Manual', 'White, Silver, Red, Blue', 750000, '2020-08-20', 'Active'),
        ('Ford', 'Figo', 'Titanium Plus', 'Car', '1.2L', 'Petrol', 'Automatic', 'White, Silver, Black, Red', 850000, '2020-08-20', 'Active'),
        ('Ford', 'Aspire', 'Titanium', 'Car', '1.2L', 'Petrol', 'Manual', 'White, Black, Silver, Gold', 950000, '2021-03-10', 'Active'),
        ('Ford', 'Aspire', 'Titanium Plus', 'Car', '1.5L', 'Diesel', 'Manual', 'White, Black, Silver, Blue', 1080000, '2021-03-10', 'Active'),
        ('Ford', 'Endeavour', 'Titanium', 'Car', '2.0L', 'Diesel', 'Automatic', 'White, Black, Silver, Brown', 3500000, '2020-11-05', 'Active'),
        ('Ford', 'Endeavour', 'Titanium Plus', 'Car', '3.2L', 'Diesel', 'Automatic', 'White, Black, Silver, Blue', 3800000, '2020-11-05', 'Active'),
        ('Ford', 'Freestyle', 'Titanium', 'Car', '1.2L', 'Petrol', 'Manual', 'White, Silver, Red, Orange', 820000, '2022-01-12', 'Active'),
        ('Ford', 'Mustang', 'GT', 'Car', '5.0L', 'Petrol', 'Automatic', 'Red, Black, White, Yellow', 7500000, '2021-12-01', 'Active'),
        ('Royal Enfield', 'Classic 350', 'Standard', 'Bike', '349cc', 'Petrol', 'Manual', 'Black, Chrome, Desert Storm', 180000, '2019-05-15', 'Active'),
        ('Royal Enfield', 'Classic 350', 'Halcyon', 'Bike', '349cc', 'Petrol', 'Manual', 'Black, Green, Maroon', 185000, '2020-02-28', 'Active'),
        ('Royal Enfield', 'Bullet 350', 'Standard', 'Bike', '346cc', 'Petrol', 'Manual', 'Black, Silver, Royal Blue', 165000, '2021-07-20', 'Active'),
        ('Royal Enfield', 'Himalayan', 'Standard', 'Bike', '411cc', 'Petrol', 'Manual', 'Granite Black, Snow White, Rock Red', 210000, '2020-09-18', 'Active'),
        ('Royal Enfield', 'Interceptor 650', 'Standard', 'Bike', '648cc', 'Petrol', 'Manual', 'Orange Crush, Baker Express, Ventura Storm', 290000, '2022-04-10', 'Active'),
        ('Royal Enfield', 'Continental GT 650', 'Standard', 'Bike', '648cc', 'Petrol', 'Manual', 'Ventura Storm, Mr. Clean, Ice Queen', 310000, '2021-01-25', 'Active'),
        ('Royal Enfield', 'Meteor 350', 'Fireball', 'Bike', '349cc', 'Petrol', 'Manual', 'Fireball Red, Fireball Yellow', 195000, '2020-10-30', 'Active'),
        ('Royal Enfield', 'Meteor 350', 'Stellar', 'Bike', '349cc', 'Petrol', 'Manual', 'Stellar Black, Stellar Blue', 200000, '2022-02-14', 'Active'),
        ('Royal Enfield', 'Hunter 350', 'Retro', 'Bike', '349cc', 'Petrol', 'Manual', 'Rebel Black, Rebel Blue, Rebel Red', 175000, '2021-11-08', 'Active'),
        ('Royal Enfield', 'Scram 411', 'Standard', 'Bike', '411cc', 'Petrol', 'Manual', 'Graphite Red, Graphite Blue, Graphite Yellow', 205000, '2021-05-20', 'Active')
    ]
    
    cursor.executemany('''
        INSERT INTO vehicles (brand, model_name, variant, vehicle_type, engine_capacity, fuel_type, transmission, color_options, base_price, launch_date, status)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', vehicle_data)
    
    conn.commit()
    print("✅ Vehicle data inserted successfully!")

insert_vehicle_data()


✅ Vehicle data inserted successfully!


In [5]:
def insert_customer_data():
    """Insert sample customer data"""
    first_names = ['Rahul', 'Priya', 'Amit', 'Sneha', 'Vikram', 'Kavya', 'Arjun', 'Neha', 'Rohit', 'Anita', 
                   'Sanjay', 'Deepika', 'Manoj', 'Shreya', 'Karan', 'Pooja', 'Gaurav', 'Ritika', 'Abhishek', 'Ravi']
    
    last_names = ['Sharma', 'Patel', 'Kumar', 'Singh', 'Agarwal', 'Gupta', 'Joshi', 'Mehta', 'Reddy', 'Nair',
                  'Tiwari', 'Desai', 'Banerjee', 'Ghosh', 'Jain', 'Pandey', 'Rao', 'Iyer', 'Chopra', 'Malhotra']
    
    cities = ['Mumbai', 'Delhi', 'Bangalore', 'Chennai', 'Hyderabad', 'Pune', 'Ahmedabad', 'Kolkata', 'Jaipur', 'Indore']
    states = ['Maharashtra', 'Delhi', 'Karnataka', 'Tamil Nadu', 'Telangana', 'Maharashtra', 'Gujarat', 'West Bengal', 'Rajasthan', 'Madhya Pradesh']
    
    customer_data = []
    for i in range(20):
        customer_data.append((
            first_names[i],
            last_names[i],
            f'+91-{9000000000 + i}',
            f'{first_names[i].lower()}.{last_names[i].lower()}{i}@email.com',
            f'{random.randint(100, 999)} Street, Area {random.randint(1, 10)}',
            random.choice(cities),
            random.choice(states),
            f'199{random.randint(0, 5)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}',
            f'202{random.randint(1, 3)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}',
            random.choice(['New', 'Returning'])
        ))
    
    cursor.executemany('''
        INSERT INTO customers (first_name, last_name, phone, email, address, city, state, date_of_birth, registration_date, customer_type)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', customer_data)
    
    conn.commit()
    print("✅ Customer data inserted successfully!")

insert_customer_data()


✅ Customer data inserted successfully!


In [6]:
def insert_employee_data():
    """Insert sample employee data"""
    positions = ['Sales Executive', 'Service Advisor', 'Technician', 'Sales Manager', 'Service Manager', 'Cashier', 'Receptionist']
    
    employee_data = []
    for i in range(1, 21):
        showroom_id = random.randint(1, 20)
        position = random.choice(positions)
        base_salary = {
            'Sales Executive': random.randint(25000, 35000),
            'Service Advisor': random.randint(30000, 40000),
            'Technician': random.randint(20000, 30000),
            'Sales Manager': random.randint(45000, 60000),
            'Service Manager': random.randint(50000, 65000),
            'Cashier': random.randint(20000, 25000),
            'Receptionist': random.randint(18000, 25000)
        }
        
        employee_data.append((
            showroom_id,
            f'Employee{i}',
            f'LastName{i}',
            position,
            f'+91-{8000000000 + i}',
            f'employee{i}@company.com',
            f'202{random.randint(0, 3)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}',
            base_salary[position],
            round(random.uniform(0.01, 0.05), 3),
            'Active'
        ))
    
    cursor.executemany('''
        INSERT INTO employees (showroom_id, first_name, last_name, position, phone, email, hire_date, salary, commission_rate, status)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', employee_data)
    
    conn.commit()
    print("✅ Employee data inserted successfully!")

insert_employee_data()


✅ Employee data inserted successfully!


In [7]:
def insert_inventory_data():
    """Insert sample inventory data"""
    inventory_data = []
    
    for i in range(1, 21):
        showroom_id = random.randint(1, 20)
        vehicle_id = random.randint(1, 20)
        stock_qty = random.randint(5, 50)
        reserved_qty = random.randint(0, 5)
        available_qty = stock_qty - reserved_qty
        
        inventory_data.append((
            showroom_id,
            vehicle_id,
            stock_qty,
            reserved_qty,
            available_qty,
            random.randint(3, 10),
            datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        ))
    
    cursor.executemany('''
        INSERT INTO inventory (showroom_id, vehicle_id, stock_quantity, reserved_quantity, available_quantity, reorder_level, last_updated)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', inventory_data)
    
    conn.commit()
    print("✅ Inventory data inserted successfully!")

insert_inventory_data()


✅ Inventory data inserted successfully!


In [8]:
def insert_sales_data():
    """Insert sample sales data"""
    payment_methods = ['Cash', 'Card', 'Bank Transfer', 'Finance', 'Cheque']
    sale_statuses = ['Completed', 'Pending', 'Delivered']
    
    sales_data = []
    for i in range(1, 21):
        sale_price = random.randint(150000, 5000000)
        discount = random.randint(5000, 50000)
        final_amount = sale_price - discount
        sale_date = datetime.now() - timedelta(days=random.randint(1, 365))
        delivery_date = sale_date + timedelta(days=random.randint(7, 30))
        
        sales_data.append((
            random.randint(1, 20),  # showroom_id
            random.randint(1, 20),  # customer_id
            random.randint(1, 20),  # vehicle_id
            random.randint(1, 20),  # salesperson_id
            sale_date.strftime('%Y-%m-%d'),
            sale_price,
            discount,
            final_amount,
            random.choice(payment_methods),
            delivery_date.strftime('%Y-%m-%d'),
            random.choice(sale_statuses)
        ))
    
    cursor.executemany('''
        INSERT INTO sales (showroom_id, customer_id, vehicle_id, salesperson_id, sale_date, sale_price, discount_amount, final_amount, payment_method, delivery_date, sale_status)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', sales_data)
    
    conn.commit()
    print("✅ Sales data inserted successfully!")

insert_sales_data()


✅ Sales data inserted successfully!


In [9]:
def insert_remaining_data():
    """Insert sample data for remaining tables"""
    
    # Test Drives Data
    test_drive_data = []
    statuses = ['Scheduled', 'Completed', 'Cancelled']
    times = ['09:00', '10:30', '14:00', '16:30']
    
    for i in range(20):
        test_drive_data.append((
            random.randint(1, 20),  # customer_id
            random.randint(1, 20),  # vehicle_id
            random.randint(1, 20),  # showroom_id
            (datetime.now() + timedelta(days=random.randint(1, 30))).strftime('%Y-%m-%d'),
            random.choice(times),
            random.choice([30, 45, 60]),
            random.choice(statuses),
            f'Test drive feedback {i+1}' if random.choice([True, False]) else None,
            random.choice([0, 1])
        ))
    
    cursor.executemany('''
        INSERT INTO test_drives (customer_id, vehicle_id, showroom_id, scheduled_date, scheduled_time, duration, status, feedback, converted_to_sale)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', test_drive_data)
    
    # Expenses Data
    expense_categories = ['Rent', 'Utilities', 'Marketing', 'Maintenance', 'Salaries', 'Insurance']
    expense_data = []
    
    for i in range(20):
        expense_data.append((
            random.randint(1, 20),  # showroom_id
            random.choice(expense_categories),
            f'Expense description {i+1}',
            random.randint(5000, 100000),
            (datetime.now() - timedelta(days=random.randint(1, 90))).strftime('%Y-%m-%d'),
            f'Manager{random.randint(1, 5)}',
            f'RCP{1000+i}'
        ))
    
    cursor.executemany('''
        INSERT INTO expenses (showroom_id, expense_category, description, amount, expense_date, approved_by, receipt_number)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', expense_data)
    
    # Targets Data
    target_data = []
    periods = ['Monthly', 'Quarterly', 'Yearly']
    types = ['Sales', 'Revenue']
    
    for i in range(20):
        target_value = random.randint(100000, 5000000)
        achieved_value = random.randint(50000, target_value)
        
        target_data.append((
            random.randint(1, 20),  # showroom_id
            random.randint(1, 20),  # employee_id
            random.choice(periods),
            random.choice(types),
            target_value,
            achieved_value,
            '2024-01-01',
            '2024-12-31'
        ))
    
    cursor.executemany('''
        INSERT INTO targets (showroom_id, employee_id, target_period, target_type, target_value, achieved_value, start_date, end_date)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', target_data)
    
    conn.commit()
    print("✅ Test drives, expenses, and targets data inserted successfully!")

insert_remaining_data()


✅ Test drives, expenses, and targets data inserted successfully!


In [10]:
def display_database_summary():
    """Display summary of all tables"""
    tables = ['showrooms', 'vehicles', 'customers', 'employees', 'inventory', 
              'sales', 'test_drives', 'expenses', 'targets']
    
    print("📊 DATABASE SUMMARY")
    print("=" * 50)
    
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        print(f"{table.capitalize():15}: {count:3} records")
    
    print("\n🎯 SAMPLE DATA PREVIEW")
    print("=" * 50)
    
    # Show sample showrooms data
    print("\n🏢 SHOWROOMS:")
    df_showrooms = pd.read_sql_query("SELECT showroom_name, brand, city, manager_name FROM showrooms LIMIT 5", conn)
    print(df_showrooms.to_string(index=False))
    
    # Show sample vehicles data
    print("\n🚗 VEHICLES:")
    df_vehicles = pd.read_sql_query("SELECT brand, model_name, variant, base_price FROM vehicles LIMIT 5", conn)
    print(df_vehicles.to_string(index=False))
    
    # Show sample sales data
    print("\n💰 RECENT SALES:")
    df_sales = pd.read_sql_query("""
        SELECT s.sale_date, sh.showroom_name, v.model_name, s.final_amount, s.sale_status
        FROM sales s
        JOIN showrooms sh ON s.showroom_id = sh.showroom_id
        JOIN vehicles v ON s.vehicle_id = v.vehicle_id
        ORDER BY s.sale_date DESC
        LIMIT 5
    """, conn)
    print(df_sales.to_string(index=False))

display_database_summary()


📊 DATABASE SUMMARY
Showrooms      :  20 records
Vehicles       :  20 records
Customers      :  20 records
Employees      :  20 records
Inventory      :  20 records
Sales          :  20 records
Test_drives    :  20 records
Expenses       :  20 records
Targets        :  20 records

🎯 SAMPLE DATA PREVIEW

🏢 SHOWROOMS:
showroom_name brand      city manager_name
Ford Downtown  Ford    Mumbai Rajesh Kumar
 Ford Highway  Ford     Delhi Priya Sharma
 Ford Central  Ford Bangalore   Amit Patel
   Ford Elite  Ford   Chennai Sunita Reddy
 Ford Premium  Ford Hyderabad Vikram Singh

🚗 VEHICLES:
brand model_name       variant  base_price
 Ford   EcoSport      Titanium   1150000.0
 Ford   EcoSport Titanium Plus   1280000.0
 Ford       Figo      Titanium    750000.0
 Ford       Figo Titanium Plus    850000.0
 Ford     Aspire      Titanium    950000.0

💰 RECENT SALES:
 sale_date           showroom_name  model_name  final_amount sale_status
2025-07-15     Royal Enfield Royal Classic 350      717510.0   D

In [11]:
def quick_analysis():
    """Perform quick analysis on the database"""
    print("📈 QUICK BUSINESS ANALYSIS")
    print("=" * 50)
    
    # Sales by brand
    print("\n💼 Sales by Brand:")
    df_brand_sales = pd.read_sql_query("""
        SELECT v.brand, COUNT(*) as total_sales, AVG(s.final_amount) as avg_sale_amount
        FROM sales s
        JOIN vehicles v ON s.vehicle_id = v.vehicle_id
        GROUP BY v.brand
    """, conn)
    print(df_brand_sales.to_string(index=False))
    
    # Top performing showrooms
    print("\n🏆 Top Performing Showrooms:")
    df_top_showrooms = pd.read_sql_query("""
        SELECT sh.showroom_name, sh.brand, COUNT(*) as total_sales, SUM(s.final_amount) as total_revenue
        FROM sales s
        JOIN showrooms sh ON s.showroom_id = sh.showroom_id
        GROUP BY s.showroom_id
        ORDER BY total_revenue DESC
        LIMIT 5
    """, conn)
    print(df_top_showrooms.to_string(index=False))
    
    # Inventory status
    print("\n📦 Current Inventory Status:")
    df_inventory = pd.read_sql_query("""
        SELECT v.brand, SUM(i.available_quantity) as total_available, 
               COUNT(*) as models_in_stock
        FROM inventory i
        JOIN vehicles v ON i.vehicle_id = v.vehicle_id
        GROUP BY v.brand
    """, conn)
    print(df_inventory.to_string(index=False))

quick_analysis()


📈 QUICK BUSINESS ANALYSIS

💼 Sales by Brand:
        brand  total_sales  avg_sale_amount
         Ford            9     2.432773e+06
Royal Enfield           11     2.019871e+06

🏆 Top Performing Showrooms:
       showroom_name         brand  total_sales  total_revenue
Royal Enfield Legend Royal Enfield            4     11338033.0
        Ford Highway          Ford            4      7625633.0
        Ford Express          Ford            2      4995787.0
       Ford Downtown          Ford            1      4244309.0
        Ford Central          Ford            1      3311115.0

📦 Current Inventory Status:
        brand  total_available  models_in_stock
         Ford              238               11
Royal Enfield              244                9


In [ ]:
# Commit all changes and close connection
conn.commit()
print("✅ All data successfully saved to database!")
print(f"📁 Database file: {DB_NAME}")
print(f"📊 Total tables created: 9")
print("🔗 Connection will remain open for further operations")

# Uncomment the line below if you want to close the connection
conn.close()

✅ All data successfully saved to database!
📁 Database file: showroom_management.db
📊 Total tables created: 9
🔗 Connection will remain open for further operations


### Test the TEXT TO SQL BOT

In [1]:
import os
from src.utils import config
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_groq import ChatGroq

In [2]:
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING", "false").lower() == "true"
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")

os.environ["LANGCHAIN_TRACING_V2"] = str(LANGSMITH_TRACING).lower()
os.environ["LANGCHAIN_API_KEY"] = LANGSMITH_API_KEY if LANGSMITH_API_KEY else ""
os.environ["LANGCHAIN_PROJECT"] = LANGSMITH_PROJECT if LANGSMITH_PROJECT else ""
os.environ["LANGCHAIN_ENDPOINT"] = LANGSMITH_ENDPOINT if LANGSMITH_ENDPOINT else ""

In [3]:
db = SQLDatabase.from_uri(
    "sqlite:///L:\RoadMapToSwitch\Text_To_SQL-Agentic_Bot\src\db\showroom_management.db"
)
llm = ChatGroq(model=config.GROQ_MODEL_NAME, api_key=config.GROQ_API_KEY, temperature=0)

<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\swata\AppData\Local\Temp\ipykernel_35436\2205312503.py:2: SyntaxWarning: invalid escape sequence '\R'
  "sqlite:///L:\RoadMapToSwitch\Text_To_SQL-Agentic_Bot\src\db\showroom_management.db"


In [4]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [5]:
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001E2F7E14AA0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001E2F7E14AA0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001E2F7E14AA0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [6]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

['dialect', 'top_k']


In [7]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)
print(system_message)

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see w

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, toolkit.get_tools(), prompt=system_message)

In [12]:
example_query = "Give me the list of employees who birthday is in August"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Give me the list of employees who birthday is in August
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (95dssa1ar)
 Call ID: 95dssa1ar
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

customers, employees, expenses, inventory, sales, service_records, showrooms, targets, test_drives, vehicles
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (kjsgzyjw9)
 Call ID: kjsgzyjw9
  Args:
    table_names: employees
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE employees (
	employee_id INTEGER, 
	showroom_id INTEGER, 
	first_name TEXT NOT NULL, 
	last_name TEXT NOT NULL, 
	position TEXT NOT NULL, 
	phone TEXT, 
	email TEXT, 
	hire_date DATE, 